In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import os
import torch
import wandb
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: 
            if k in cfg.pp.keys():  cfg.pp[k] = v
            else:                   setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)

travis_attack.data: INFO     Will load dataset rotten_tomatoes with use_small_ds set to True


In [ ]:
cfg.wandb['mode'] = 'online'
trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, optimizer,
                  ds, initial_eval=False, use_cpu=False)
print(vars(cfg))
trainer.train()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find run.


{'pp_name': 'prithivida/parrot_paraphraser_on_T5', 'dataset_name': 'rotten_tomatoes', 'sts_name': 'sentence-transformers/paraphrase-MiniLM-L12-v2', 'nli_name': 'howey/electra-small-mnli', 'vm_name': 'textattack/distilbert-base-uncased-rotten-tomatoes', 'seed': 420, 'use_small_ds': True, 'sampling_strategy': 'sample', 'lr': 4e-05, 'reward_fn': 'reward_fn_contradiction_and_letter_diff', 'reward_clip_max': 3, 'reward_clip_min': 0, 'reward_base': 0, 'reward_vm_multiplier': 12, 'sts_threshold': 0.6, 'contradiction_threshold': 0.2, 'pp_letter_diff_threshold': 30, 'reward_penalty_type': 'ref_logp', 'kl_coef': 0.2, 'ref_logp_coef': 0.01, 'pp': {'do_sample': True, 'min_length': 4, 'max_length': 48, 'temperature': 1, 'length_penalty': 1, 'top_p': 1, 'repetition_penalty': 1}, 'orig_max_length': 32, 'pin_memory': True, 'zero_grad_with_none': False, 'pad_token_embeddings': False, 'embedding_padding_multiple': 8, 'orig_padding_multiple': 8, 'bucket_by_length': True, 'shuffle_train': False, 'remove_m

travis_attack.trainer: INFO     Now on epoch 1 of 5


travis_attack.trainer: INFO     Now on epoch 2 of 5


travis_attack.trainer: INFO     Now on epoch 3 of 5


travis_attack.trainer: INFO     Now on epoch 4 of 5


travis_attack.trainer: INFO     Now on epoch 5 of 5


In [ ]:
# ## TO RESUME RUN
# cfg = Config()
# cfg.run_id = '2jq83fdx'
# cfg.run_name = "pleasant-wind-125"
# cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
# run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()

travis_attack.insights: INFO     Dataframes have shapes ['training_step: (145, 48)', 'train: (145, 32)', 'valid: (30, 32)', 'test: (2, 32)']
travis_attack.insights: INFO     Adding text metrics for column orig_l


travis_attack.insights: INFO     Adding text metrics for column pp_l


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


travis_attack.insights: INFO     Adding text metrics for column orig_l


travis_attack.insights: INFO     Adding text metrics for column pp_l


...

travis_attack.insights: INFO     Adding text metrics for column pp_l


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs


travis_attack.insights: INFO     Adding text metrics for column orig_l


travis_attack.insights: INFO     Adding text metrics for column pp_l


travis_attack.insights: INFO     Calculating metric differences between orig and pp
travis_attack.insights: INFO     Calculating text pair statistics for (orig, pp) unique pairs
